In [2]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import QuantileTransformer

from whatlies.language import BytePairLanguage, UniversalSentenceLanguage

2022-05-06 15:15:40.060097: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-06 15:15:40.060120: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

df = pd.read_csv("data/annotations.csv")[['text', 'excitement']].drop_duplicates()

In [4]:
%%time 

model_cv = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))
model_cv.fit(list(df['text']), df['excitement'])

CPU times: user 9.85 s, sys: 27.8 s, total: 37.6 s
Wall time: 3.67 s


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [5]:
%%time

model_bp = make_pipeline(BytePairLanguage("en"), QuantileTransformer(), LogisticRegression(max_iter=1000))
model_bp.fit(list(df['text']), df['excitement'])

CPU times: user 21 s, sys: 11.9 s, total: 32.9 s
Wall time: 9.02 s


Pipeline(steps=[('bytepairlanguage', BytePairLanguage(lang='en')),
                ('quantiletransformer', QuantileTransformer()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [6]:
%%time

model_use = make_pipeline(UniversalSentenceLanguage(), QuantileTransformer(), LogisticRegression(max_iter=1000))
model_use.fit(list(df['text']), df['excitement'])

2022-05-06 15:15:54.354018: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-06 15:15:54.354187: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-06 15:15:54.354196: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-06 15:15:54.354211: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-05-06 15:15:54.354347: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

CPU times: user 5min 12s, sys: 54.7 s, total: 6min 7s
Wall time: 1min 31s


Pipeline(steps=[('tfhublanguage',
                 TFHubLanguage(url='https://tfhub.dev/google/universal-sentence-encoder/4')),
                ('quantiletransformer', QuantileTransformer()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [7]:
import pandas as pd
from doubtlab.ensemble import DoubtEnsemble
from doubtlab.reason import ProbaReason, ShortConfidenceReason, LongConfidenceReason, DisagreeReason

y = df['excitement']

# Calc the predictions
pred_cv = model_cv.predict(list(df['text']))
pred_bp = model_bp.predict(list(df['text']))
pred_use = model_use.predict(list(df['text']))

# Calc the confidence values
proba_cv = model_cv.predict_proba(list(df['text']))
proba_bp = model_bp.predict_proba(list(df['text']))
proba_use = model_use.predict_proba(list(df['text']))

In [13]:
df_predicate = pd.DataFrame(dict(
    proba_cv=ProbaReason.from_proba(proba_cv),
    proba_bp=ProbaReason.from_proba(proba_bp),
    proba_use=ProbaReason.from_proba(proba_use),
    short_cv=ShortConfidenceReason.from_proba(proba_cv, y, classes=model_cv.classes_, threshold=0.2),
    short_bp=ShortConfidenceReason.from_proba(proba_bp, y, classes=model_bp.classes_, threshold=0.2),
    short_use=ShortConfidenceReason.from_proba(proba_use, y, classes=model_use.classes_, threshold=0.2),
    long_cv=ShortConfidenceReason.from_proba(proba_cv, y, classes=model_cv.classes_, threshold=0.6),
    long_bp=ShortConfidenceReason.from_proba(proba_bp, y, classes=model_bp.classes_, threshold=0.6),
    long_use=ShortConfidenceReason.from_proba(proba_use, y, classes=model_use.classes_, threshold=0.6),
    disagree_bp=DisagreeReason.from_pred(pred_cv, pred_bp),
    disagree_use=DisagreeReason.from_pred(pred_cv, pred_use)
))

In [16]:
df_predicate.sort_values("long_cv", ascending=False).head(5)

,proba_cv,proba_bp,proba_use,short_cv,short_bp,short_use,long_cv,long_bp,long_use,disagree_bp,disagree_use
61931,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
28038,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
28013,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
28014,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
42314,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
21008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from collections import Counter

summary = df_predicate.sum(axis=1).value_counts().reset_index().sort_values("index")
summary.columns = ["overlap", "count"]
summary

,overlap,count
0,0.0,56555
4,1.0,388
6,2.0,276
5,3.0,337
3,4.0,798
1,5.0,1786
2,6.0,1699
7,7.0,64
8,8.0,29


In [28]:
idx = df_predicate.loc[(df_predicate.sum(axis=1) >= 4)].index
df_check = df.iloc[idx]
df_check.to_json("doubt.jsonl", orient="records", lines=True)

In [72]:
df_orig = pd.read_csv("data/annotations.csv")

In [91]:
df_orig.loc[lambda d: d['text'].isin(df_check['text'])].head(20)

,text,id,rater_id,example_very_unclear,excitement
461,Reading this made me pretty damn happy. Congrats hope it works out for you two.,eeqn6py,46,False,0
769,"[NAME] would be an exciting guy to bring here. I think he'd at some help at RB, especially if [NAME] ain't interested",efdy564,81,False,1
4074,Oh mate! Happy days. haha. Yeah I totally thought you were starting a course of Radiotherapy haha. All good,ee39hkt,24,False,0
5021,Woo hoo!!! Free at last!,ef1bvtq,78,False,1
5735,"“Oh hey, I think a second ago [NAME] just put a move on [NAME]! Oh boy I wonder how that was.”",edm0c9b,40,False,0
5797,Let's get things back on track! Happy new year!,eczc44a,5,False,1
6421,"This feels more like ""Lofi Rock"" if that exists, or even Jazz. Very interesting.",ed4l5sx,76,False,0
6965,That would be crazy premature. But would be awesome later when results are out and the stories behind the scenes of the [NAME] team emerges.,ef3oy07,51,False,1
7224,"Woah, woah, woah. You're trying to tell me that elites have a stranglehold in a country with an *actual* aristocracy?!",edfxx6i,21,False,0
8780,Yay!!! Finally the fix we're all looking for after hours of endless scrolling.,edwiv4a,62,False,1


In [76]:
ml_texts = df_check['text']

disagree_texts = (df_orig
 .groupby("text")
 .agg(unclear=("example_very_unclear", "sum"), 
      label=("excitement","sum"))
 .loc[lambda d: (d['unclear'] != 0) | (d['label'].isin([1, 2]))]
 .reset_index()['text'])

In [82]:
print(f"# disagreements set {ml_texts.shape[0]}")
print(f"# ml set {disagree_texts.shape[0]}\n")
print(f"# disagreements also in ml set {disagree_texts.isin(ml_texts).sum()}")
print(f"# ml also in disagreement set {(~ml_texts.isin(disagree_texts)).sum()}")

# disagreements set 5377
# ml set 6706

# disagreements also in ml set 4130
# ml also in disagreement set 554


In [75]:
df_disagree

,text,unclear,label
0,">it'll get invaded by tankie, unfortunately. How apropos.",1,0
1,">sexuality shouldn’t be a grouping category It makes you different from othet ppl so imo it fits the definition of ""grouping""",2,0
2,Everybody clap your hands 👏👏👏👏👏👏👏👏 Sliiiide to the left. 🕺,2,0
3,"I don’t give a F F the haters F all the haters Much love, no hate Machines within, machines within",2,0
4,Lol! ♿♿♿ &#x200B;,1,0
...,...,...,...
6701,😘 ☂ ☂️☂️,2,0
6702,😵 when 😤 mommy 🤨 gets 👀 the 🤔 🅱️endies 😫 just 🔥🔥🔥 right 👌,1,0
6703,🤣 wow... I read it all in a serious tone until the end! 🤣,0,2
6704,"🤣 yes... sure. Let me talk to a teenage Q believer, listening to [NAME] who doesent even know what projection is... next",1,0


It seems like we have 55 candidates that were "suspicious" that did not pop up in the labeller disagreement set. 

In [65]:
df_check[(~texts.isin(df_disagree['text']))]

,text,excitement
638,LHW is such a barren shithole that I’d be more excited for this than anyone at 205 getting a shot,1
1921,"Ohhhh, that makes sense. Cheers mate",1
4006,Excited for season 3 too. Winter is getting too cold,1
5797,Let's get things back on track! Happy new year!,1
6294,I'll have a look and see if I can find it. Cheers for the info!,1
7020,I'm pretty excited to see how [NAME] develops. Appears that he could be a decent enough player in the future.,1
9025,Happy Birthday! ✨,0
9948,"I’m excited to see [NAME] come in here. I hope he plays some more quality, aggressive minutes",1
11253,"Fam, I fucking love [NAME] play. Gotta be the most exciting dude in the league other than [NAME] when he’s hitting everything from 30ft+",1
12824,"I did, I did!!!! Now I feel special",1


In [29]:
from prodigy.components.db import connect

db = connect()
dataset = db.get_dataset("go-emotions")

In [30]:
pd.DataFrame(dataset)[['text', 'excitement', 'answer']].groupby(['excitement', 'answer']).count()

text
excitement answer      
0          accept     5
           ignore     2
           reject     3
1          accept    49
           ignore    10
           reject    93

<br><br><br><br><br><br><br><br><br><br><br><br>

In [58]:
df = pd.read_csv("data/annotations.csv")

df[df['text'] == "Happy Birthday!!!"]

,text,id,rater_id,example_very_unclear,excitement
76392,Happy Birthday!!!,eed81k3,10,False,0
171527,Happy Birthday!!!,eed81k3,73,False,0
185904,Happy Birthday!!!,eed81k3,30,False,0


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [53]:
(df
 .groupby("text")
 .agg(n_label = ("excitement", "sum"), 
      unclear=("example_very_unclear", "sum"))
 .reset_index()
 .loc[lambda d: d['text'].str.contains("Birthday")]
)

,text,n_label,unclear
11296,"Happy Birthday I hope this means anything, the little things matter sometimes.",1,0
11297,Happy Birthday [NAME]!,2,0
11298,Happy Birthday [NAME]. You wont read this message but itll be here.,1,0
11299,Happy Birthday!,2,0
11300,Happy Birthday! ✨,3,0
11301,Happy Birthday!!,2,0
11302,Happy Birthday!! It was great seeing her at the Royal Rumble and SmackDown.,1,0
11303,Happy Birthday!!!,0,0
21490,I'm sorry that happened to you. You planned a awesome day that I wouldve loved celebrating with you. Happy Birthday,1,0
25553,I’m sorry. Happy Birthday! I care ❤️,0,0


In [60]:
df[df['text'].str.contains('boop snoot')]

,text,id,rater_id,example_very_unclear,excitement
83963,OMG THOSE TINY SHOES! *desire to boop snoot intensifies*,eei8tja,41,False,0
90458,OMG THOSE TINY SHOES! *desire to boop snoot intensifies*,eei8tja,15,False,0
123640,OMG THOSE TINY SHOES! *desire to boop snoot intensifies*,eei8tja,8,False,0
141487,OMG THOSE TINY SHOES! *desire to boop snoot intensifies*,eei8tja,61,True,0
210655,OMG THOSE TINY SHOES! *desire to boop snoot intensifies*,eei8tja,16,False,0
